In [1]:
from fluid import Fluid
import numpy as np

from scipy.optimize import fsolve

co2 = Fluid('co2')

In [2]:
# none of these values are real, they are just filler values

#https://en.wikipedia.org/wiki/List_of_thermal_conductivities

r_casing = (5.563 / 2.) * .0254 # m
k_casing = 32.2 # [W/mK] 

r_concrete = (5 / 2.)  * .0254 
k_concrete = 0.8 

r_tube = (2.375 / 2.) * .0254
k_tube = 32.2 

r_internal = (2 / 2.) * .0254 

pipe_e = 0.0015 # TODO find something real

def pipe_ctr(r_o, r_i, k, L): # https://www.engineersedge.com/heat_transfer/heat_loss_insulated_pipe_13865.htm
    return np.log(r_o/r_i)/(2*np.pi*k*L)

def total_pipe_ctr(r_casing, r_concrete, r_tube, k_casing, k_concrete, k_tube, L):
    tr_casing = pipe_ctr(r_casing, r_concrete, k_casing,L)
    tr_concrete = pipe_ctr(r_concrete, r_tube, k_concrete, L)
    tr_tube = pipe_ctr(r_tube, r_internal, k_tube, L)

    tr_tot = tr_casing + tr_concrete + tr_tube

    return tr_tot

tr_pipe = lambda length: total_pipe_ctr(r_casing, r_concrete, r_tube, k_casing, k_concrete, k_tube, length)

def conduction_resistance(k, t, a):
    return t/(k*a)

def darcy_friction_factor(Re, e_d):
    if Re < 2000:
        return 64/Re
    elif Re < 4000:
        return 0.316/Re**0.25
    else:
        return .0055*[1+((2*10**4)*(e_d) + 10**6/Re)**(1/3)] # 0.25/np.log10(e_d/3.7 + 5.74/Re**0.9)**2 # 

def head_loss(v, mu, l, d):
    Re = v*d/mu
    e_d = pipe_e / d
    f = darcy_friction_factor(Re, e_d)
    return f*l*v**2/(2*d*9.81)

Because of conservation of energy, the energy balance for a control volume can be written as:
$$
Q_{in}  = \dot m \big[ (h_2 - h_1) + \frac{1}{2}(v_2^2 - v_1^2) + g(z_2-z_1) + g h_l \big] 
$$

In [3]:
class State:
    def __init__(self,h,z,rho):
        self.h = h
        self.z = z
        self.rho = rho

In [4]:
def heat_transfer(state2, state1, mdot, head_loss):
    qin = mdot * ((state2.h - state1.h) + 0.5 * (state2.v**2 - state1.v**2) + 9.81 * (state2.z - state1.z) + g * head_loss)
    return qin

In [ ]:
def solve_finite_element_method(mdot, state_in, length, T_external, radius, ):

    v_in = mdot / (state_in.rho * np.pi * radius**2)
    head_loss = head_loss(v_in,co2.viscosity(D=state_in.rho,H=state_in.h) , length,2*radius)
    T_in = co2.temperature(D=state_in.rho,H=state_in.h)
    q_in = tr_pipe(length) * (T_external - T_in) / mdot
    # mechanical energy
    me_lhs = state_in.p / state_in.rho + 0.5 * v_in**2 + 9.81 * state_in.z
    
    # thermal energy
    u1 = state_in.h - state_in.p / state_in.rho
    
    z_out = state_in.z - length
    head_loss = 1
    def f(rho_out, h_out):
        p_out = co2.pressure(D = rho_out, H = h_out)
        v_out = state_in.rho * p_out / rho_out
        u_out = h_out - p_out / rho_out
        me_loss = p_out / rho_out + 0.5 *(v_out)**2 + 9.81 * z_out + 9.81 * head_loss - me_lhs
        heat_loss = u1 + q_in + 9.81 * head_loss - u_out
        return me_loss**2, heat_loss**2
    
    # solve for rho_out, h_out
    rho_out, h_out = fsolve(f, [state_in.rho, state_in.h])
    return State(h_out, z_out, rho_out)
    
    